## Introduction


In this lab, you will learn how to leverage pre-trained models to build image classifiers instead of building a model from scratch.

## Import Libraries and Packages

Let's start the lab by importing the libraries that we will be using in this lab.

First, we will import the ImageDataGenerator module since we will be leveraging it to train our model in batches.

In [6]:
from keras.preprocessing.image import ImageDataGenerator

In [7]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense

we will be leveraging the ResNet50 model to build our classifier

In [8]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

<a id='item32'></a>

## Download Data

In [ ]:

!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

And now if you check the left directory pane, you should see the zipped file *concrete_data_week3.zip* appear. So, let's go ahead and unzip the file to access the images. Given the large number of images in the dataset, this might take a couple of minutes, so please be patient, and wait until the code finishes running.

In [ ]:
!unzip concrete_data_week3.zip

## Define Global Constants


1. We are obviously dealing with two classes, so *num_classes* is 2. 
2. The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3. We will training and validating the model using batches of 100 images.

In [9]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100

<a id='item34'></a>

## Construct ImageDataGenerator Instances

In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to *preprocess_input* which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.

In [10]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

Next, we will use the *flow_from_directory* method to get the training images as follows:

In [11]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week3/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [12]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week3/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')


Found 10001 images belonging to 2 classes.


<a id='item35'></a>

## Build, Compile and Fit Model

In [13]:
model = Sequential()

Adding the ResNet50 pre-trained model to our model. 
We don't want to include the top layer or the output layer of the pre-trained model. 
Define our own output layer and train it so that it is optimized for our image dataset. 
In order to leave out the output layer of the pre-trained model, set the argument *include_top* to **False**.

In [14]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))


94658560/94653016 [==============================] - 28s 0us/step


Defining our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.

In [15]:
model.add(Dense(num_classes, activation='softmax'))

You can access the model's layers using the *layers* attribute of our model object. 

In [16]:
model.layers

The model is composed of two sets of layers. 
The first set is the layers pertaining to ResNet50 and the second set is a single layer, which is our Dense layer that we defined above.

Accessing the ResNet50 layers:

In [17]:
model.layers[0].layers

Since the ResNet50 model has already been trained, training the ResNet part is not required, we have to train our dense output layer

In [18]:
model.layers[0].trainable = False

In [19]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


Compiling our model using the **adam** optimizer.

In [21]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=1,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    verbose=1,
)


Epoch 1/1
301/301 [==============================] - 8266s 27s/step - loss: 0.0447 - accuracy: 0.9856 - val_loss: 7.3358e-04 - val_accuracy: 0.9060


Since training can take a long time when building deep learning models, it is always a good idea to save your model

In [24]:
model.save('classifier_resnet_model.h5')